Copyright © 2025 Technical University of Denmark

In [1]:
import numpy as np
from lcg_plus.base import State
from lcg_plus.operations.symplectic import squeezing

In [2]:
#Check that photon-number moment calculation works on a squeezed state
state = State(1)
r = 1
state.apply_symplectic(squeezing(r,0 ))

print(state.get_photon_number_moments())
print('nbar', np.sinh(r)**2)
print('nvar', 2 * np.sinh(r)**2*np.cosh(r)**2)

(1.3810978455418155, 6.57705820900412)
nbar 1.3810978455418155
nvar 6.577058209004121


In [3]:
#Photon number moment of a coherent state
state = State(1)
alpha = 0.5
state.apply_displacement(np.array([alpha*np.sqrt(2*state.hbar),0]))
print(state.get_photon_number_moments())
print(alpha**2)

(0.25, 0.24999999999999994)
0.25


In [4]:
#Calculating the photon-number moments in the coherent state decomposition

from lcg_plus.states.nongauss import prepare_fock_coherent

#Fock state 
ns = np.arange(30)
for n in ns:
    fock = prepare_fock_coherent(n, inf = 1e-10, fast = True) 
    ex, var = fock.get_photon_number_moments()
    print(ex.real, var.real)

1.0000000827403705e-10 9.999999999999996e-11
1.0000000001991225 1.0000000001987743
2.0000000000316818 2.000000000037667
3.000000000912946 3.000000000895086
4.000000000441295 4.000000000441288
5.000000000490432 5.000000000490432
6.000000000923478 6.000000000923478
7.0000000008946826 7.0000000008946826
8.000000000937657 8.000000000937657
9.000000000937037 9.000000000937037
10.000000001091172 10.00000000109577
11.000000001208567 11.000000001208567
12.000000001293273 12.000000001293273
13.00000000139701 13.00000000139701
14.000000001500947 14.000000001500947
15.000000001600178 15.000000001600178
16.000000001700105 16.000000001700105
17.000000001799837 17.000000001799837
18.000000001900148 18.000000001900105
19.00000000199999 19.00000000199999
20.000000002099902 20.000000002099902
21.00000000219985 21.00000000219985
22.00000000230013 22.00000000230013
23.000000002400096 23.000000002400096
24.000000002500006 24.000000002500006
25.00000000259999 25.00000000259999
26.000000002700013 26.0000000

In [5]:
#Apply Gaussian ops on a Fock state

n = 10
eta = 0.8
r = 0.5
state = prepare_fock_coherent(n, inf = 1e-6, fast = True) 
state.apply_loss(np.array([eta]), np.zeros(1))

state.apply_symplectic(squeezing(r,0))
ex, var = state.get_photon_number_moments()
print(ex)
print(var)
print(np.sqrt(var))

(12.616198975025634+17.67975840871381j)
(30.78814755872905+44.44649315737482j)
(6.513702093008666+3.4117689543309364j)


In [6]:
#Check with strawberryfields
import strawberryfields as sf
from strawberryfields.ops import Sgate, BSgate, MeasureFock, MeasureHomodyne, Fock, Catstate, LossChannel

In [7]:
eng = sf.Engine('fock', backend_options={"cutoff_dim":300})
prog = sf.Program(1)


with prog.context as q:
    Fock(n) | q[0]
    LossChannel(eta) | q[0]
    Sgate(r,0) | q[0]
    
result = eng.run(prog)

state = result.state
ex, var = state.mean_photon(0)
print(ex)
print(var)
print(np.sqrt(var))

12.616185395929595
55.324706191576496
7.4380579583367386
